# 본문 스크래핑

In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument('headless') # 창 숨기기


In [2]:
# from user_agent import generate_user_agent, generate_navigator

# print(generate_user_agent(device_type='desktop'))
# print(generate_user_agent(os='win', device_type='desktop'))

# navigator = generate_navigator()

# random_user = generate_user_agent(os='win', device_type='desktop')
# print(random_user)

In [3]:
from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


In [4]:
url_df = pd.read_csv("./나무문화/2022/나무문화_링크_2022_7월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(2867, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/yangyoseop91/222835899184,주간일기 8: 파리 2,2022.07.31.
1,https://blog.naver.com/kuna611/222835896356,"[블챌] 7월 4주차_주간 일기? 일상 기록이라 쓰고, 욕망의...",2022.07.31.
2,https://blog.naver.com/y_girok_/222835898249,[주간일기 챌린지 / 7월 4주차] 7월 25일 ~ 7월 31일,2022.07.31.
3,https://blog.naver.com/roseyh78/222835897663,경주 건천맛집 명문갈비 가야밀면 편백나무숲 맛집,2022.07.31.
4,https://blog.naver.com/anuyheel/222835899414,[수영구 남천동 / 광안리 화실] 보통날의 작업실 - 아크릴...,2022.07.31.
...,...,...,...
2862,https://blog.naver.com/babjule/222796851268,"무더위 이태원 데이트코스 추천! 리움미술관, 우육미엔...",2022.07.01.
2863,https://blog.naver.com/tjdrud34/222796850531,베트남 푸꾸옥 투어 버스 썬월드 혼똔섬 케이블카 시간 및...,2022.07.01.
2864,https://blog.naver.com/dnpftlsh/222796844700,"6월4주 월미도 당일여행 / 월미바다열차, 월미산정상",2022.07.01.
2865,https://blog.naver.com/crazyboy/222796842010,김지하 시(詩) 7 - 사편 1,2022.07.01.


In [5]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


2867


In [6]:
# df = pd.DataFrame(columns = ['url','본문'])
# df.to_csv("./나무문화/2022/나무문화_본문_2022_7월.csv", index=False, encoding='UTF-8')
# df.to_excel("./나무문화/2022/나무문화_본문_2022_7월.xlsx", index=False, encoding='UTF-8')

In [7]:
df = pd.read_csv("./나무문화/2022/나무문화_본문_2022_7월.csv", encoding='UTF-8')
df

,url,본문
0,https://blog.naver.com/yangyoseop91/222835899184,인트로\n살려달라고 써놓고 부랴부랴 수정하는 중... 이번주 주간일기도 마저 써야되...
1,https://blog.naver.com/kuna611/222835896356,#25일급여 #삼성카드 #특판적금 #서울행복신협 #구즉신협 #카카오뱅크8%적금 #새...
2,https://blog.naver.com/y_girok_/222835898249,안녕-!\n재미없었고 별 다른 일이 없어서\n조용했던 저번주와는 달리\n시끌벅적 우...
3,https://blog.naver.com/roseyh78/222835897663,경주 켄싱턴리조트 퇴실 후 곧장 집으로 가자하면 어쩌나 했는데\n다행이 내가 가고 ...
4,https://blog.naver.com/anuyheel/222835899414,수영구에서 뭐 재미있는 거 배울만한 게 있을까?\n새로운 것을 도전해보면서 문화 생...
...,...,...
1195,https://blog.naver.com/hoperecipe/222834415619,광주 산책 광주천 동구와 남구를 가르는 하천이에요\n전날 518민주광장에서 야경을 ...
1196,https://blog.naver.com/jnanoori/222834414816,"코로나19 재유행에 대비하여\n미추홀구\n호흡기환자 진료센터 및\n코로나 검사, 진..."
1197,https://blog.naver.com/yangklee/222834413045,"티티키\n산류천석, 산에서 흐르는 물이 바위를 뚫는다.\n천안 동남권에 위치한다. ..."
1198,https://blog.naver.com/busan_suyeong/222828321611,#\n'수영구'와\n'수영구 도시재생 현장지원센터'\n그리고\n'숲 우리들'이 함께...


In [8]:
len(df)

1200

In [9]:
#url_df.iloc[5349]

In [10]:
# 이건 필요할때 쓰면 됨. 블로그가 닫혔거나 삭제되었을때 빈 본문을 가진 데이터프레임 추가.
# content_dic = {'url' : ["https://blog.naver.com/himynet/222036795571"], '본문' : [None]}
# additional_df = pd.DataFrame(content_dic)
# df = df.append(additional_df, ignore_index = True)

In [11]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []
waitsec = random.uniform(4,4.5)

from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(100)
    time.sleep(waitsec)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2022/나무문화_본문_2022_7월.csv", index=False, encoding='UTF-8')
    #df.to_excel("./나무문화/2020/나무문화_본문_2020_7월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    
    display.clear_output(wait=True) # 출력지우기
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


1430 1430


C:\Users\user\AppData\Local\Temp\ipykernel_2424\3956159720.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)


In [ ]:
df

In [ ]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))
